<a href="https://colab.research.google.com/github/YuktiMohan/Tensorflow-in-Practice/blob/master/Lesson%203%3A%20Natural%20Language%20Processing/3B%3A%20Word%20Embedding/IMDB_reviews_dataset_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
!pip install tensorflow 

In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


Note:
1. If tensorflow version is 1.x, then execute command: tf.enable_eager_execution(). This is enabled by default in tensorflow 2.x.
2. To install tensorflow datasets use command: !pip install -q tensorflow-datasets
3. Tensorflow Data Services (TFDS): It is a library which contains many datasets in lots of different categories. 

In [0]:
#Importing IMDB review dataset 

import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCP5BI0/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCP5BI0/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCP5BI0/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

train, test = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

Reference for the following block of code: https://www.tensorflow.org/tutorials/customization/basics

This code generates a list of sentences(reviews) and labels for training and testing data.

In [0]:
for s,l in train:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  

In [0]:
#To convert the list generated above to numpy arrays

final_training_labels = np.array(training_labels)
final_testing_labels = np.array(testing_labels)

In [0]:
#List of Hyperparameters:

vocab_size = 10000
embedding_dim = 16
trunc_type = 'post'
max_len = 120
oov_tok = "<007>" 

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(training_sequences, maxlen = max_len, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_len, truncating = trunc_type)

In [0]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_len),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
                            ])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
new_model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_len),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
                            ])

new_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, final_training_labels, epochs = num_epochs, validation_data=(testing_padded, final_testing_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 222us/sample - loss: 0.4989 - accuracy: 0.7364 - val_loss: 0.3783 - val_accuracy: 0.8312
Epoch 2/10
25000/25000 [==============================] - 5s 189us/sample - loss: 0.2410 - accuracy: 0.9070 - val_loss: 0.4102 - val_accuracy: 0.8198
Epoch 3/10
25000/25000 [==============================] - 5s 181us/sample - loss: 0.0899 - accuracy: 0.9780 - val_loss: 0.4972 - val_accuracy: 0.8127
Epoch 4/10
25000/25000 [==============================] - 5s 181us/sample - loss: 0.0220 - accuracy: 0.9973 - val_loss: 0.6150 - val_accuracy: 0.8060
Epoch 5/10
25000/25000 [==============================] - 5s 188us/sample - loss: 0.0053 - accuracy: 0.9997 - val_loss: 0.6722 - val_accuracy: 0.8111
Epoch 6/10
25000/25000 [==============================] - 5s 185us/sample - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.7295 - val_accuracy: 0.8114
Epoch 7/10
25000/25000 [==========================

In [0]:
new_model.fit(padded, final_training_labels, epochs=num_epochs, validation_data=(testing_padded, final_testing_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5s 199us/sample - loss: 0.6037 - accuracy: 0.6913 - val_loss: 0.5001 - val_accuracy: 0.8091
Epoch 2/10
25000/25000 [==============================] - 5s 182us/sample - loss: 0.4143 - accuracy: 0.8560 - val_loss: 0.4153 - val_accuracy: 0.8308
Epoch 3/10
25000/25000 [==============================] - 5s 185us/sample - loss: 0.3237 - accuracy: 0.8893 - val_loss: 0.3926 - val_accuracy: 0.8350
Epoch 4/10
25000/25000 [==============================] - 5s 194us/sample - loss: 0.2715 - accuracy: 0.9092 - val_loss: 0.3972 - val_accuracy: 0.8328
Epoch 5/10
25000/25000 [==============================] - 5s 198us/sample - loss: 0.2357 - accuracy: 0.9215 - val_loss: 0.4204 - val_accuracy: 0.8268
Epoch 6/10
25000/25000 [==============================] - 4s 179us/sample - loss: 0.2089 - accuracy: 0.9326 - val_loss: 0.4493 - val_accuracy: 0.8217
Epoch 7/10
25000/25000 [==========================

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

#Ouput = (Vocab_size, embedding_dim)

(10000, 16)
